In [1]:
!pip install mdp

     |████████████████████████████████| 460kB 7.1MB/s 


In [2]:
 from google.colab import drive
 import sys
 drive.mount('/content/drive')
 sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks')
 import mdp

Mounted at /content/drive


In [3]:
transition_probs = {
    's0': {
        'a0': {'s0': 0.4, 's4': 0.2, 's1':0.1,'s2':0.1,'s3':0.1,'s5':0.1},
        'a1': {'s1': 0.4, 's5':0.2,'s2':0.1,'s3':0.1,'s4':0.1,'s0':0.1},
        'a2': {'s2': 0.4, 's3': 0.2, 's0':0.1,'s1':0.1,'s4':0.1,'s5':0.1}
    },
    's1': {
        'a0': {'s0': 0.4, 's4': 0.2, 's1':0.1,'s2':0.1,'s3':0.1,'s5':0.1},
        'a1': {'s1': 0.4, 's5':0.2,'s2':0.1,'s3':0.1,'s4':0.1,'s0':0.1},
        'a2': {'s2': 0.4, 's3': 0.2, 's0':0.1,'s1':0.1,'s4':0.1,'s5':0.1}
    },
    's2': {
        'a0': {'s0': 0.4, 's4': 0.2, 's1':0.1,'s2':0.1,'s3':0.1,'s5':0.1},
        'a1': {'s1': 0.4, 's5':0.2,'s2':0.1,'s3':0.1,'s4':0.1,'s0':0.1},
        'a2': {'s2': 0.4, 's3': 0.2, 's0':0.1,'s1':0.1,'s4':0.1,'s5':0.1}
    },
    's3': {
        'a0': {'s0': 0.4, 's4': 0.2, 's1':0.1,'s2':0.1,'s3':0.1,'s5':0.1},
        'a1': {'s1': 0.4, 's5':0.2,'s2':0.1,'s3':0.1,'s4':0.1,'s0':0.1},
        'a2': {'s2': 0.4, 's3': 0.2, 's0':0.1,'s1':0.1,'s4':0.1,'s5':0.1}
    },
    's4': {
        'a0': {'s0': 0.4, 's4': 0.2, 's1':0.1,'s2':0.1,'s3':0.1,'s5':0.1},
        'a1': {'s1': 0.4, 's5':0.2,'s2':0.1,'s3':0.1,'s4':0.1,'s0':0.1},
        'a2': {'s2': 0.4, 's3': 0.2, 's0':0.1,'s1':0.1,'s4':0.1,'s5':0.1}
    },
    's5': {
        'a0': {'s0': 0.4, 's4': 0.2, 's1':0.1,'s2':0.1,'s3':0.1,'s5':0.1},
        'a1': {'s1': 0.4, 's5':0.2,'s2':0.1,'s3':0.1,'s4':0.1,'s0':0.1},
        'a2': {'s2': 0.4, 's3': 0.2, 's0':0.1,'s1':0.1,'s4':0.1,'s5':0.1}
    }
}
rewards = {
    's0': {'a0': {'s0': 5}},
    's0': {'a0': {'s4': -5}},
    's0': {'a1': {'s1': -5}},
    's0': {'a2': {'s3': -5}},
    's1': {'a0': {'s0': 5}},
    's1': {'a0': {'s4': -5}},
    's1': {'a1': {'s1': -5}},
    's1': {'a2': {'s3': -5}},
    's2': {'a0': {'s0': 5}},
    's2': {'a0': {'s4': -5}},
    's2': {'a1': {'s1': -5}},
    's2': {'a2': {'s3': -5}},
    's3': {'a0': {'s0': 5}},
    's3': {'a0': {'s4': -5}},
    's3': {'a1': {'s1': -5}},
    's3': {'a2': {'s3': -5}},
    's4': {'a0': {'s0': 5}},
    's4': {'a0': {'s4': -5}},
    's4': {'a1': {'s1': -5}},
    's4': {'a2': {'s3': -5}},
    's5': {'a0': {'s0': 5}},
    's5': {'a0': {'s4': -5}},
    's5': {'a1': {'s1': -5}},
    's5': {'a2': {'s3': -5}}

}

from mdp import MDP
mdp = MDP(transition_probs, rewards, initial_state='s0')
import numpy as np

We can now use MDP just as any other gym environment:

In [4]:
print('initial state =', mdp.reset())
next_state, reward, done, info = mdp.step('a1')
print('next_state = %s, reward = %s, done = %s' % (next_state, reward, done))

initial state = s0
next_state = s5, reward = 0.0, done = False


but it also has other methods that you'll need for Value Iteration

In [5]:
print("mdp.get_all_states =", mdp.get_all_states())
print("mdp.get_possible_actions('s0') = ", mdp.get_possible_actions('s1'))
print("mdp.get_next_states('s0', 'a1') = ", mdp.get_next_states('s1', 'a0'))
print("mdp.get_reward('s1', 'a0', 's0') = ", mdp.get_reward('s1', 'a0', 's0'))
print("mdp.get_transition_prob('s0', 'a1', 's0') = ", mdp.get_transition_prob('s1', 'a0', 's0'))

mdp.get_all_states = ('s0', 's1', 's2', 's3', 's4', 's5')
mdp.get_possible_actions('s0') =  ('a0', 'a1', 'a2')
mdp.get_next_states('s0', 'a1') =  {'s0': 0.4, 's4': 0.2, 's1': 0.1, 's2': 0.1, 's3': 0.1, 's5': 0.1}
mdp.get_reward('s1', 'a0', 's0') =  0.0
mdp.get_transition_prob('s0', 'a1', 's0') =  0.4


## **Policy Iteration**




In [6]:
# Compute the Q-value using the formula above
def get_action_value(mdp, state_values, state, action, gamma):

    # Initialise Q 
    Q = 0  
    for s in mdp.get_all_states():
      # Compute Q using the equation above
      Q = Q + mdp.get_transition_prob(state, action[s], s)*(mdp.get_reward(state, action[s], s) + gamma*state_values[s]) 
    return Q

In [7]:
# Compute the Q-value using the formula above
def get_action_value_max(mdp, state_values, state, action, gamma):

    # Initialise Q 
    Q = 0  
    for s in mdp.get_all_states():
      # Compute Q using the equation above
      Q = Q + mdp.get_transition_prob(state, action, s)*(mdp.get_reward(state, action, s) + gamma*state_values[s]) 
    return Q

In [8]:
def policy_eval(mdp, state_value_dict, state_action_dict, state_act_val):
  import random
  #Intialization

  for s in mdp.get_all_states():
    state_value_dict[s] = 5
    a = mdp.get_possible_actions(s)
    state_action_dict[s] = random.sample(a, 1)[0]
    print(state_action_dict)
    

  
  for i in range(0, 100):
    delta = 0
    for s in mdp.get_all_states():
      v = state_value_dict[s]
      V = get_action_value(mdp,state_value_dict, s, state_action_dict, 0.9)
      delta =  max( delta, abs(v - V))
      print("iter %4i   |   delta: %6.5f   |   " % (i, delta), end="")
      print('   '.join("V(%s) = %.3f" % (s, v) for s, v in state_value_dict.items()))
      state_value_dict[s] = V

    if delta <= 0.01:
      print('terminated....')
      break
  
  return state_value_dict, state_action_dict




  
  

In [9]:
def policy_improv(mdp, state_value_dict, state_action_dict, state_act_val):
  policy_stable = True
  for s in mdp.get_all_states():
    old_action = state_action_dict[s]
    
    # recomputer π∗(s) using new values from policy_eval
    #i = 0
    V = []
    A = []
    for a in mdp.get_possible_actions(s):
      V.append(get_action_value_max(mdp, state_value_dict, s, a, 0.9))
      A.append(a)
      #i += 1
    new_action = A[np.argmax(V)]

    if old_action != new_action:
      policy_stable = False
    
    state_act_val[s] = {A[np.argmax(V)]:V[np.argmax(V)]}
  return policy_stable, state_act_val
    
    




In [10]:
policy_stable = False
state_act_val = {}
state_value_dict = {}
state_action_dict = {}
while policy_stable == False:
  state_value_dict, state_action_dict = policy_eval(mdp,state_value_dict,state_action_dict, state_act_val)
  policy_stable, _ = policy_improv(mdp, state_value_dict, state_action_dict, state_act_val)
  print('policy stable? ->',policy_stable)




Streaming output truncated to the last 5000 lines.
iter   18   |   delta: 530.60996   |   V(s0) = 1596.119   V(s1) = 1111.911   V(s2) = 1241.078   V(s3) = 1385.302   V(s4) = 1465.819   V(s5) = 1508.425
iter   18   |   delta: 592.46291   |   V(s0) = 1596.119   V(s1) = 1642.521   V(s2) = 1241.078   V(s3) = 1385.302   V(s4) = 1465.819   V(s5) = 1508.425
iter   18   |   delta: 661.52603   |   V(s0) = 1596.119   V(s1) = 1642.521   V(s2) = 1833.541   V(s3) = 1385.302   V(s4) = 1465.819   V(s5) = 1508.425
iter   18   |   delta: 700.08291   |   V(s0) = 1596.119   V(s1) = 1642.521   V(s2) = 1833.541   V(s3) = 2046.828   V(s4) = 1465.819   V(s5) = 1508.425
iter   18   |   delta: 720.48500   |   V(s0) = 1596.119   V(s1) = 1642.521   V(s2) = 1833.541   V(s3) = 2046.828   V(s4) = 2165.902   V(s5) = 1508.425
iter   19   |   delta: 762.47829   |   V(s0) = 1596.119   V(s1) = 1642.521   V(s2) = 1833.541   V(s3) = 2046.828   V(s4) = 2165.902   V(s5) = 2228.910
iter   19   |   delta: 784.69872   |   V(s0

In [ ]:
if has_graphviz:
    display(plot_graph_with_state_values(mdp, state_value_dict))

In [12]:
print("Final state values:", state_value_dict)

Final state values: {'s0': 0.04939369203728545, 's1': 0.04614350993168626, 's2': 0.04538443117039274, 's3': 0.04463783955120653, 's4': 0.04390352965576268, 's5': 0.04245906923405449}


# Value Iteration

In [13]:
# Compute the Q-value using the formula above
def get_action_value_v(mdp, state_values, state, action, gamma):

    # Initialise Q 
    Q = 0  
    for s in mdp.get_all_states():
      # Compute Q using the equation above
      Q = Q + mdp.get_transition_prob(state, action, s)*(mdp.get_reward(state, action, s) + \
                                                               gamma*state_values[s]) 
    return Q

def get_new_state_value_v(mdp, state_values, state, gamma):
    import numpy as np

    # Computes next V(s) as in formula above. Do not change state_values in process.
    if mdp.is_terminal(state):
        return 0

    # Initialise the dict
    A = [a for a in mdp.get_possible_actions(state)]
    v = np.zeros(len(mdp.get_possible_actions(state)))
    i = 0
    
    # Compute all possible options
    for a in mdp.get_possible_actions(state):
      v[i] = get_action_value_v(mdp, state_values, state, a, gamma)
      A[i] = a
      i = i+1

    # Recover V(s) and π∗(s) as per the formula above
    V = {A[np.argmax(v)]:v[np.argmax(v)]}
    
    return V

In [ ]:
# parameters
gamma = 0.9            # discount for MDP
num_iter = 100         # maximum iterations, excluding initialization
# stop VI if new values are this close to old values (or closer)
min_difference = 0.001

# initialize V(s)
state_values = {s: 0 for s in mdp.get_all_states()}



for i in range(num_iter):

    # Compute new state values using the functions defined above.
    # It must be a dict {state : float V_new(state)}
    new_state_values = {}
    for s in mdp.get_all_states():
      nsv = get_new_state_value_v(mdp, state_values, s, gamma)
      a = list(nsv)[0]
      v = nsv[a]
      new_state_values[s] = v

    assert isinstance(new_state_values, dict)

    # Compute difference
    diff = max(abs(new_state_values[s] - state_values[s])
               for s in mdp.get_all_states())
    print("iter %4i   |   diff: %6.5f   |   " % (i, diff), end="")
    print('   '.join("V(%s) = %.3f" % (s, v) for s, v in state_values.items()))
    state_values = new_state_values

    if diff < min_difference:
        print("Terminated")
        break

In [ ]:
if has_graphviz:
    display(plot_graph_with_state_values(mdp, state_values))

In [16]:
print("Final state values:", state_value_dict)

Final state values: {'s0': 0.04939369203728545, 's1': 0.04614350993168626, 's2': 0.04538443117039274, 's3': 0.04463783955120653, 's4': 0.04390352965576268, 's5': 0.04245906923405449}
